In [12]:
from tensorflow import keras
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [2]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)
            

2021-09-24 05:55:32.993255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-24 05:55:32.993287: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-24 05:55:32.993304: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kris-IdeaPad): /proc/driver/nvidia/version does not exist
2021-09-24 05:55:32.993623: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
for layer in vgg19.layers:
    layer.trainable = False

In [4]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

In [5]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [7]:
# Define the batch size:
batch_size=32

# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory='./datasets/hotdog/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./datasets/hotdog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [9]:
model.fit(train_generator, steps_per_epoch=15, 
                    epochs=5, validation_data=valid_generator, 
                    validation_steps=15)

Epoch 1/5
15/15 [==============================] - 86s 6s/step - loss: 0.5425 - accuracy: 0.7446 - val_loss: 0.5588 - val_accuracy: 0.7333
Epoch 2/5
15/15 [==============================] - 86s 6s/step - loss: 0.4344 - accuracy: 0.8069 - val_loss: 0.4612 - val_accuracy: 0.7812
Epoch 3/5
15/15 [==============================] - 85s 6s/step - loss: 0.4139 - accuracy: 0.7961 - val_loss: 0.5139 - val_accuracy: 0.7563
Epoch 4/5
15/15 [==============================] - 87s 6s/step - loss: 0.3999 - accuracy: 0.8229 - val_loss: 0.4997 - val_accuracy: 0.7625
Epoch 5/5
15/15 [==============================] - 89s 6s/step - loss: 0.3441 - accuracy: 0.8433 - val_loss: 0.4579 - val_accuracy: 0.7792


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
flattened (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
predictions (Dense)          (None, 2)                 50178     
Total params: 20,074,562
Trainable params: 50,178
Non-trainable params: 20,024,384
_________________________________________________________________


In [11]:
model.save('./models/hotdog_vgg.h5')

In [13]:
loaded_model = keras.models.load_model('./models/hotdog_vgg.h5')

In [16]:
import os

DATASETS = './datasets'
HOTDOG_DIR = os.path.join(DATASETS, 'hotdog')
TEST_DIR = os.path.join(HOTDOG_DIR, 'test')
TRAIN_DIR = os.path.join(HOTDOG_DIR, 'train')

In [18]:
TRAIN_DIR, TEST_DIR

('./datasets/hotdog/train', './datasets/hotdog/test')

In [55]:
import numpy as np
from PIL import Image

def img_to_numpy(img, resize=True):
    image = Image.open(img)
    if resize:
        image = image.resize((224, 224))
    np_image = np.array(image)
    return np_image

def load_images(data, targets):
    path = os.path.join(data, targets)
    files = list(os.listdir(path))
    images = list()
    for f in files[:1]:
        file_path = os.path.join(path, f)
        images.append(img_to_numpy(file_path))

    return images


In [57]:
pos_images = load_images(TEST_DIR, 'hot_dog')
neg_images = load_images(TEST_DIR, 'not_hot_dog')

pos_np = np.array(pos_images)
neg_np = np.array(neg_images)

In [58]:
loaded_model.predict(pos_np)

array([[0.9816425 , 0.01835755],
       [0.7685794 , 0.23142058],
       [0.9728262 , 0.02717385],
       [0.9016454 , 0.09835456],
       [0.99896955, 0.0010305 ]], dtype=float32)

In [59]:
loaded_model.predict(neg_np)

array([[0.03280906, 0.967191  ],
       [0.01999859, 0.9800014 ],
       [0.4022279 , 0.59777206],
       [0.09397189, 0.9060281 ],
       [0.67313814, 0.32686183]], dtype=float32)